# Blackjack

In [207]:
import random
import numpy as np
import time

class BlackJack: 
    def __init__(self, seed=0): 
        self.deck = [i for i in range(52)]
        random.seed(seed)
        random.shuffle(self.deck)
        self.dealer = []
        self.player = []

    def set_state(self, state): 
        (deck, dealer, player, seed) = state

        self.deck = deck
        self.dealer = dealer
        self.player = player
        random.seed(seed)
        random.shuffle(self.deck)

    def convert_card(self, card): 
        value = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A'][card % 13]
        suit = 'CDHS'[card // 13]
        return f'{value}{suit}'

    def get_state(self): 
        return self.deck, self.dealer, self.player, self.seed

    def draw_player(self):
        self.player.append(self.deck.pop(0))

    def draw_dealer(self): 
        self.dealer.append(self.deck.pop(0))

    def calc(self, player = True): 
        my_deck = self.player if player else self.dealer

        has_ace = False
        value = 0
        for card in my_deck:
            index = card % 13
            if index == 0:
                has_ace = True
                value += 11
            elif index >= 10:
                value += 10
            else:
                value += index + 1

        if has_ace and value <= 11:
            value += 10

        if value > 21:
            value = -1

        return value 
    
    def get_result(self):
        playerscore, dealerscore = self.calc(player = True), self.calc(player = False)
        return np.sign(playerscore - dealerscore)
        

    def display_game(self, move = '', elapsed_time = 1):
        time.sleep(elapsed_time)
        print(f'Remaining deck: {len(self.deck)}')
        if move != '':
            print(move)
        print(f'Dealer: {list(map(self.convert_card,self.dealer))}, Value: {self.calc(player = False)}')
        print(f'Player: {list(map(self.convert_card,self.player))}, Value: {self.calc(player = True)}')
        print('')


In [208]:
def Player(env):
    return np.random.choice(['hit', 'stick'])

def Dealer(env):
    if env.get_result() > 0 and env.calc(player = False) != -1:
        return 'hit'
    else:
        return 'stick'
         

## Game Loop

In [ ]:
bj = BlackJack(seed = random.randint(0, 1000))
# bj = BlackJack(seed = 0)

bj.draw_player()
bj.draw_player()
bj.draw_dealer()
bj.display_game()

while bj.calc(player = True) != -1:
    action = Player(bj)
    print(f'Player {action.lower()}s!')
    if action == 'hit':
        bj.draw_player()
        bj.display_game()
    else:
        break
    

bj.draw_dealer()
bj.display_game()

while bj.calc(player = False) != -1:
    action = Dealer(bj)
    print(f'Dealer {action.lower()}s!')
    if action == 'hit':
        bj.draw_dealer()
        bj.display_game()
    else:
        break

result = bj.get_result()
if result == 0:
    print('Draw')
elif result == 1:
    print('Player wins')
else:
    print('Dealer wins')

Remaining deck: 49
Dealer: ['7S'], Value: 7
Player: ['3H', 'KC'], Value: 13

Player sticks!
Remaining deck: 48
Dealer: ['7S', '3S'], Value: 10
Player: ['3H', 'KC'], Value: 13

Dealer hits!
Remaining deck: 47
Dealer: ['7S', '3S', 'KH'], Value: 20
Player: ['3H', 'KC'], Value: 13

Dealer sticks!
Dealer wins
